# Natural Language Processing

## Questions

1. What are the pros and cons of using a stateful RNN versus a stateless RNN?

- Stateless Recurrent Neural Networks start with a hidden state full of zeros on each iteration, then it updates this state at each time step, and after the last, it throws it away as it is not nieeded anymore.  If the RNN is instructed to preserve this finalstate after processing a trainning batch, it could use it as initial state in the next batch and learn more long-term-patterns, despite only backpropagating through short sequences. That is a stateful RNN. It only is useful if each input sequence in a batch starts exacly where the corresponding sequence in the previous batch let off.
- Stateless RNNs can only capture patterns whose length is less than, or equal to, the size of the windows the RNN is trained on. Conversely, stateful RNNs can capture longer-term patterns. However, implementing a stateful RNN is much harder⁠—especially preparing the dataset properly. Moreover, stateful RNNs do not always work better, in part because consecutive batches are not independent and identically distributed (IID). Gradient Descent is not fond of non-IID datasets.


2. Why do people use encoder–decoder RNNs rather than plain sequence-to-sequence RNNs for automatic translation?
- It is necesary to process the sentence and then translate, if it is not done it misses information from the future of the sentence. For example, if you want to translate 'I like ice cream' or 'That is like when i grew up in Uruguay', the word like has a different meaning that can only be taken into account if the whole sentence is proecessed before generating a translation. A sequence-to-sequence RNN reads one word and translates it, then the other and so on, so the encoder decoder architecture was introduced to process all the sentence, and then translate.

3. How can you deal with variable-length input sequences? What about variable-length output sequences?
- With variable length input sequences a mask and padding can be used, that implies that al the id 0 embeddings are ignored by the model. So if the sequence is shorter than the sequence max length, there is no problem. But, if the sequence is larger, there is no strategy possible. For outputs, the model outputs an end of the result that controls the length of the output, so if it is shorter than the maximum length trained it automatically shortens the output. Ragged tensors can hold sequences of variable lengths, and Keras now supports them, which simplifies handling variable-length input sequences (at the time of this writing, it still does not handle ragged tensors as targets on the GPU, though). 

4. What is beam search, and why would you use it? What tool can you use to implement it?
Beam search is a strategy to select the next word of an output sequence. For each new word, it considers the top-k best sentences and calculates the width-k new phrases and their probability of beign the right one. Implementing it requires writing a custom memory cell.

5. What is an attention mechanism? How does it help?
When processing long sequences, sometimes a word meaning is important even before it is used in the translation. Attention is a technique that allows the decoder of an Encoder-Decoder model to focus on the appropiate words at each time step as encoded by the encoder. It computes a weighted sum of all the encoder outputs, determining which words the decoder will focus at the step. Each decoder cell receives the hidden state from the previous time step plus the result of the alignment model, that calculates an alignment score for each input time step. The mechanism works using a dense layer that has a single neuron that processes all the encoders output with the decoder previous hidden state. The output is a score for each of the encoders output. Given a n lenght sentence of input and output. The model computes n2 weights. 

6. What is the most important layer in the transformer architecture? What is its purpose?
The multi head attention is the most important layer, it is a layer that allows to interpret multiple meanings of a word by dividing the sequence, model dimension matrix of words into several matrixes, appling attention with multiple sub queries, sub keys and sub values, each of them multiplied by a weight matrix before attention and then after the algorithm we have sub head as the name of the layer says. After computing all these meanings for each word, the layer concatenates them resulting in a matrix that has information about the multiple meanings, the position and the relationship of the word with the other tokens of the sentence in many ways. This way, the layer outputs a matrix of sentence, embedding size dimension that has a lot of meaning information.

7. When would you need to use sampled softmax?
When the vocabulary is large, in the final step of the transformer the softmax function outputs a probability for a matrix that is of size (sequence, vocabulary_size), outputed by a linear layer that outputs a score of confidence of the model for each word in the vocabulary.
So applying softmax to get probabilities can be really expensive if the vocabulary is long enough. One solution is to look only at the top logits output by the model for the correct word and for a random sample of incorrect words, then compute an approximation of the loss based only on these logits. This sampled softmax is used during training to speed up the calculation of loss estimating cross-entropy loss.